連結伺服器與基本套件

In [ ]:
! pip install line-bot-sdk
! pip install flask-ngrok
! pip install django
! pip install beautifulsoup4
! pip install requests

In [ ]:
# from google.colab import drive
# import os

# drive.mount('/content/drive')
# googleDrive = '/content/drive/MyDrive'
# print(os.listdir(googleDrive))

In [ ]:
from __future__ import unicode_literals
import os
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
from flask_ngrok import run_with_ngrok
from PIL import Image
from io import StringIO
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import requests
import random
import time
import json
import math
import random
import string
import configparser
import random,string


app = Flask(__name__)
run_with_ngrok(app)
line_bot_api = LineBotApi('FQegCeIhV+PZmvMwgqYrtLGfFumHIM1IQ0pCzyNCe3lBp/r2llyjHoQEEgJhp1/EvFoUKGTtlu/5vaEYOzHU9fBhn/CNWIISnQNc94JCQZtjn8c4vHDkKWwIeZuLhCo4IIzYLAf7lkWv4hbYtor1LgdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('d1709beb6f197c641853bba06c654a74')
times = 0
Total = str()
Total2 = str()
Total3 = str()
Total4 = str()
from datetime import datetime, timezone, timedelta
tz = timezone(timedelta(hours=+8))

Locations = ['基隆市','嘉義市','台北市','嘉義縣','新北市','台南市','桃園縣','高雄市','新竹市','屏東縣','新竹縣','台東縣','苗栗縣','花蓮縣','台中市','宜蘭縣','彰化縣','澎湖縣','南投縣','金門縣','雲林縣','連江縣']
cities = ['基隆市','嘉義市','臺北市','嘉義縣','新北市','臺南市','桃園縣','高雄市','新竹市','屏東縣','新竹縣','臺東縣','苗栗縣','花蓮縣','臺中市','宜蘭縣','彰化縣','澎湖縣','南投縣','金門縣','雲林縣','連江縣']
@app.route("/callback", methods=['POST'])

def callback():
    signature = request.headers['X-Line-Signature']

    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    
    try:
        print(body,signature)
        handler.handle(body,signature)
        
    except InvalidSignatureError:
        abort(400)

    return 'OK'

def get(city): 
  token = 'CWB-88575519-8287-4BC0-AC20-45563FA82341'
  url = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001?Authorization=' + token + '&locationName=' + str(city)
  Data = requests.get(url)
  Data = (json.loads(Data.text,encoding='utf-8'))['records']

  query_res = []
  for row in Data['location']:
    if city in str(row):
      query_res.append(row)
      break
  
  res = [[]]
  for j in range(1):
    for i in query_res[0]['weatherElement']:
      res[j].append(i['time'][j]['parameter']['parameterName'])
      # print(type(i['time'][j]['parameter']['parameterName']))
  result = city + "\n天氣狀況：" + res[0][0] + "\n降雨機率：" + res[0][1] + " %\n溫度：" + res[0][2] + "~" + res[0][4] + "°C"
  return result

def get_food(Location):
  global times,Total,Total2,Total3,Total4
  Website = "https://ifoodie.tw"
  #獲取連結
  Link = list()
  #獲取店家名稱
  Name = list()
  #獲取圖片
  Photo = list()
  #獲取評分
  Rate = list()
  #均消
  Price = list()
  #評論
  Comment = list()
  #營業時間
  Time = list()
  #地址
  Address = list()
  #分類(文字雲用)
  Category = [list() for i in range(10)]
  #網站bug修復
  a = 0
  Cnum = 0
  Total = ""
  response = requests.get("https://ifoodie.tw/explore/"+Location+"/list")
  soup = BeautifulSoup(response.text, "html.parser")
  Content = soup.find("div", class_="jsx-3908823019 item-list")
  List = Content.find_all("div", class_="jsx-4282820896 restaurant-info")
  for k in List:
    # print(k)
    Link.append(Website+k.find("a").get("href"))
    Name.append(k.find("img")["alt"])
    if(a<=1):
      Photo.append(k.find("img")["src"])
    else:
      Photo.append(k.find("img")["data-src"])
    if(k.find("div", class_="jsx-1207467136 text")):
      Rate.append(k.find("div", class_="jsx-1207467136 text").getText()) 
    else:
      Rate.append("無評分")
    if (k.find("div", class_="jsx-1776651079 avg-price")):
      Price.append(k.find("div", class_="jsx-1776651079 avg-price").text)   
    else:
      Price.append("無資訊")
    Comment.append(Website+k.find("a", class_="jsx-4282820896 review-count").get("href"))
    tmp = k.find("div", class_="jsx-4282820896 info").getText()
    Time.append(tmp)
    Address.append(k.find("div", class_="jsx-4282820896 address-row").getText())
    for i in k.find_all("a", class_="jsx-1776651079 category"):
      if(i.getText()=="附近餐廳"):
        continue
      Category[Cnum].append(i.getText())
    Cnum += 1
    a += 1
  for i in range(0,10):
    num = i + 1
    category = str(Category[i])
    category = category.replace("[","")
    category = category.replace("]","")
    category = category.replace("'","")
    if(i<3):
      Total = Total+str(num)+"."+str(Name[i])+"("+category+")"+"\n"+"網站連結:"+str(Link[i])+"\n"+"圖片連結:"+str(Photo[i])+"\n"+"評分:"+str(Rate[i])+"/5"+"\n"+"均消:"+str(Price[i])+"\n"+"評論連結:"+str(Comment[i])+"\n"+"營業狀態:"+str(Time[i])+"\n"+"地址:"+str(Address[i])+"\n"      
    elif(i<6):
      Total2 = Total2+str(num)+"."+str(Name[i])+"("+category+")"+"\n"+"網站連結:"+str(Link[i])+"\n"+"圖片連結:"+str(Photo[i])+"\n"+"評分:"+str(Rate[i])+"/5"+"\n"+"均消:"+str(Price[i])+"\n"+"評論連結:"+str(Comment[i])+"\n"+"營業狀態:"+str(Time[i])+"\n"+"地址:"+str(Address[i])+"\n"      
    elif(i<9):
      Total3 = Total3+str(num)+"."+str(Name[i])+"("+category+")"+"\n"+"網站連結:"+str(Link[i])+"\n"+"圖片連結:"+str(Photo[i])+"\n"+"評分:"+str(Rate[i])+"/5"+"\n"+"均消:"+str(Price[i])+"\n"+"評論連結:"+str(Comment[i])+"\n"+"營業狀態:"+str(Time[i])+"\n"+"地址:"+str(Address[i])+"\n"
    elif(i<10):
      Total4 = Total4+str(num)+"."+str(Name[i])+"("+category+")"+"\n"+"網站連結:"+str(Link[i])+"\n"+"圖片連結:"+str(Photo[i])+"\n"+"評分:"+str(Rate[i])+"/5"+"\n"+"均消:"+str(Price[i])+"\n"+"評論連結:"+str(Comment[i])+"\n"+"營業狀態:"+str(Time[i])+"\n"+"地址:"+str(Address[i])+"\n"

@handler.add(MessageEvent,message=TextMessage)
def hello_message(event):
    if '嗨' and '你好' in event.message.text:
      reply_hello=[]
      reply_hello.append(TextSendMessage(text='哼!怎麼現在才來找我...大笨蛋'))
      reply_hello.append(TextSendMessage(text='找我有什麼事嗎'))
      line_bot_api.reply_message(event.reply_token,reply_hello)
    elif '拜託' in event.message.text:
      reply_please=[]
      reply_please.append(TextSendMessage(text=random.choice(['才不是特別為你做的呢!','只有你...才可以命令我','果然沒有我你是不行的嘛!'])))
      line_bot_api.reply_message(event.reply_token,reply_please)
    elif '可愛' in event.message.text:
      reply_cute=[]
      reply_cute.append(TextSendMessage(text=random.choice(['笨...笨蛋','不用你說我也是知道的!','真...真的嗎...少囉嗦!'])))
      line_bot_api.reply_message(event.reply_token,reply_cute)
    elif '功能' in event.message.text:
      reply_menu=[]
      reply_menu.append(TextSendMessage(text='看來你還是需要我的嘛...\n1.輸入地區即可探索美食\n2.輸入文字雲就能知道哪種食物現在最受歡迎\n3.想知道胖次的顏色\n4.想知道胖次的款式\n5.想知道我是誰\n6.想知道我的年齡\n7.想知道我的生日\n8.想知道我的興趣\n9.想跟我告白\n10.想知道當前時間\n11.想知道現在的天氣'))
      line_bot_api.reply_message(event.reply_token,reply_menu)
    elif '食物' in event.message.text[:2]:
      Location=event.message.text[3:]
      Location=Location.replace('臺','台')
      # ress=get_food(Location)
      # line_bot_api.reply_message(event.reply_token,TextSendMessage(text=ress))
      if not any(local in Location for local in Locations):
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text="查詢格式為: 食物 縣市"))
      else:
        get_food(Location)
        ress1=Total  
        print(ress1)     
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text=ress1))
        ress2=Total2   
        print(ress2)    
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text=ress2))
        ress3=Total3 
        print(ress3)     
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text=ress3))
        ress4=Total4
        print(ress4)       
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text=ress4))
    elif '胖次' and '顏色' in event.message.text:
      reply_underwear=[]
      reply_underwear.append(TextSendMessage(text=random.choice(['粉紅色','白色','黃色','黑色','紫色','裸色'])))
      line_bot_api.reply_message(event.reply_token,reply_underwear)
    elif '胖次' and '款式' in event.message.text:
      reply_style=[]
      reply_style.append(TextSendMessage(text=random.choice(['蕾絲丁字褲','安全褲','沒有穿哦','三角褲'])))
      line_bot_api.reply_message(event.reply_token,reply_style)
    elif '誰' in event.message.text:
      reply_bac=[]
      reply_bac.append(TextSendMessage(text='我是死神的第一大弟子,但是因為現代的醫學發達使我很難再難收割到人類的靈魂...只能靠萌..萌死人'))
      line_bot_api.reply_message(event.reply_token,reply_bac)
    elif '年齡' in event.message.text:
      reply_age=[]
      reply_age.append(TextSendMessage(text='哼!沒禮貌,但...你的話我是可以告訴你哦,由於是在時間出現之前已經存在的關係,因此年齡是「沒有」'))
      line_bot_api.reply_message(event.reply_token,reply_age)
    elif '生日' in event.message.text:
      reply_bir=[]
      reply_bir.append(TextSendMessage(text='4月4日...到時候一定要...幫我慶祝!'))
      line_bot_api.reply_message(event.reply_token,reply_bir)
    elif '興趣' in event.message.text:
      reply_inter=[]
      reply_inter.append(TextSendMessage(text='熱愛殺人,黑暗,RAP,黑道電影,紅酒和甜食,最討厭咖啡,人群,醫院和哈蜜瓜...一定要記得哦!'))
      line_bot_api.reply_message(event.reply_token,reply_inter)
    elif '喜歡妳' in event.message.text:
      reply_inter=[]
      reply_inter.append(TextSendMessage(text='全世界我最討♂厭♀你了...才怪...笨蛋'))
      line_bot_api.reply_message(event.reply_token,reply_inter)
    elif '幾點' in event.message.text:
      reply_time=[]
      reply_time.append(TextSendMessage(text=datetime.now(tz).strftime('這...這種問題,我當然知道,現在的時間是\n%Y年%m月%d日%H:%M:%S')))
      line_bot_api.reply_message(event.reply_token,reply_time)
    elif '天氣' in event.message.text[:2]:
      city=event.message.text[3:]
      city=city.replace('台','臺')
      if not (city in cities):
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text="查詢格式為: 天氣 縣市"))
      else:
        res=get(city)
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text=res))
    else:
      for i in event.message.text:
        reply_fail=[]
        reply_fail.append(TextSendMessage(text='囉嗦...再講清楚一點...'))
      line_bot_api.reply_message(event.reply_token,reply_fail)



# 
import os
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://502950e51d70.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Jun/2021 05:31:19] "POST /callback HTTP/1.1" 200 -


{"destination":"U0f961c7b777bb072dad4125d50cdad23","events":[]} iGzRyNvoQYiTAh3a6D+ZpZfvuG+OA8qHnG1svpD7p3k=
{"destination":"U0f961c7b777bb072dad4125d50cdad23","events":[{"type":"message","message":{"type":"text","id":"14222347859049","text":"嗨你好 死神大人"},"timestamp":1623648816279,"source":{"type":"user","userId":"U21f70e47543f9ecae44f5543731470c8"},"replyToken":"ff3f3c892ec045d3a6e90a6d55f921b3","mode":"active"}]} rFAvbdfaWWIy/lmunm+wfqd7anMAtcACdAoyb5iT870=


...

127.0.0.1 - - [14/Jun/2021 05:36:59] "POST /callback HTTP/1.1" 200 -


{"destination":"U0f961c7b777bb072dad4125d50cdad23","events":[{"type":"message","message":{"type":"text","id":"14222362068628","text":"天氣 台中市"},"timestamp":1623649025322,"source":{"type":"user","userId":"U21f70e47543f9ecae44f5543731470c8"},"replyToken":"992424b5387e482c81bccb4f25403b63","mode":"active"}]} zHb4t2wtLTw94p5oiGRK1V6ckBJ+/i9lMPDOatqjVpI=


127.0.0.1 - - [14/Jun/2021 05:37:08] "POST /callback HTTP/1.1" 200 -


{"destination":"U0f961c7b777bb072dad4125d50cdad23","events":[{"type":"message","message":{"type":"text","id":"14222364069153","text":"我喜歡妳死神大人"},"timestamp":1623649054753,"source":{"type":"user","userId":"U21f70e47543f9ecae44f5543731470c8"},"replyToken":"f02032fd71ef436f8b0209453736aa8e","mode":"active"}]} XWeu8WGhHu6QZxHKtJscyNkAYBwe5D5ugJ8xpHVAfNc=


127.0.0.1 - - [14/Jun/2021 05:37:35] "POST /callback HTTP/1.1" 200 -


In [ ]:
# import requests
# from bs4 import BeautifulSoup
# #輸入地區
# Location = str(input())

# Website = "https://ifoodie.tw"

# #獲取連結
# Link = list()
# #獲取店家名稱
# Name = list()
# #獲取圖片
# Photo = list()
# #獲取評分
# Rate = list()
# #均消
# Price = list()
# #評論
# Comment = list()
# #營業時間
# Time = list()
# #地址
# Address = list()
# #分類(文字雲用)
# Category = [list() for i in range(15)]

# Cnum = 0
# #網站bug修復
# a = 0
# #整合
# Total = str()
# response = requests.get(
#     "https://ifoodie.tw/explore/"+Location+"/list")
# soup = BeautifulSoup(response.text, "html.parser")
# Content = soup.find("div", class_="jsx-3908823019 item-list")
# List = Content.find_all("div", class_="jsx-1776651079 restaurant-info")
# for k in List:
#   # print(k)
#   Link.append(Website+k.find("a").get("href"))
#   Name.append(k.find("img")["alt"])
#   if(a<=1):
#     Photo.append(k.find("img")["src"])
#   else:
#     Photo.append(k.find("img")["data-src"])
#    if(k.find("div", class_="jsx-1207467136 text")):
    #   Rate.append(k.find("div", class_="jsx-1207467136 text").getText()) 
    #else:
    #   Rate.append("無評分")
#   if (k.find("div", class_="jsx-1776651079 avg-price")):
#     Price.append(k.find("div", class_="jsx-1776651079 avg-price").text)   
#   else:
#     Price.append("無資訊")
#   Comment.append(Website+k.find("a", class_="jsx-1776651079 review-count").get("href"))
#   tmp = k.find("div", class_="jsx-1776651079 info").getText()
#   Time.append(tmp)
#   Address.append(k.find("div", class_="jsx-1776651079 address-row").getText())
#   for i in k.find_all("a", class_="jsx-1776651079 category"):
#     if(i.getText()=="附近餐廳"):
#       continue
#     Category[Cnum].append(i.getText())
    
#   Cnum += 1
#   a += 1
# for i in range(0,15):
#   num = i + 1
#   category = str(Category[i])
#   category = category.replace("[","")
#   category = category.replace("]","")
#   category = category.replace("'","")
#   Total = Total+str(num)+"."+str(Name[i])+"("+category+")"+"\n"+"網站連結:"+str(Link[i])+"\n"+"圖片連結:"+str(Photo[i])+"\n"+"評分:"+str(Rate[i])+"/5"+"\n"+"均消:"+str(Price[i])+"\n"+"評論連結:"+str(Comment[i])+"\n"+"營業狀態:"+str(Time[i])+"\n"+"地址:"+str(Address[i])+"\n"
  
# print(Total)

In [ ]:
# 食物分類文字雲
%matplotlib inline
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image
import jieba
import numpy as np
file = open('Category.txt','w');

file.write(str(Category));

file.close()

font = "/content/drive/MyDrive/Colab Notebooks/NotoSerifCJKtc-Black.otf"
text = open('Category.txt', 'r', encoding='utf-8').read()
# 設定使用 big5 斷詞
jieba.set_dictionary("/content/drive/MyDrive/Colab Notebooks/dict.txt.big.txt")
wordlist = jieba.cut(text)
words = " ".join(wordlist)

#背景顏色預設黑色，改為白色、使用指定字體
my_wordcloud = WordCloud(background_color='white',font_path=font).generate(words)

plt.imshow(my_wordcloud)
plt.axis("off")
plt.show()
#存檔
my_wordcloud.to_file('word_cloud.png')